In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 44000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:45000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_44k_45K.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6618172503633534980-9
Page load happened
44000 https://kw.com/agent/UPA-6618172503633534980-9
***********************************
https://kw.com/agent/UPA-6661300081835241475-3
Page load happened
44001 https://kw.com/agent/UPA-6661300081835241475-3
***********************************
https://kw.com/agent/UPA-6587384954716418049-3
Page load happened
44002 https://kw.com/agent/UPA-6587384954716418049-3
***********************************
https://kw.com/agent/UPA-6587385232621277186-6
Page load happened
44003 https://kw.com/agent/UPA-6587385232621277186-6
***********************************
https://kw.com/agent/UPA-6587385257897443332-7
Page load happened
44004 https://kw.com/agent/UPA-6587385257897443332-7
***********************************
https://kw.com/agent/UPA-6694597892660834304-5
Page load happened
44005 https://kw.com/agent/UPA-6694597892660834304-5
***********************************
https://kw.com/agent/UPA-6587385193305001986-1
Page load happened
4400

Page load happened
44053 https://kw.com/agent/UPA-6823017097743900672-0
***********************************
https://kw.com/agent/UPA-6726501854363357184-2
Timeout happened no page load
44054 https://kw.com/agent/UPA-6726501854363357184-2
***********************************
https://kw.com/agent/UPA-6587385383325261824-0
Page load happened
44055 https://kw.com/agent/UPA-6587385383325261824-0
***********************************
https://kw.com/agent/UPA-6699367500158672896-0
Page load happened
44056 https://kw.com/agent/UPA-6699367500158672896-0
***********************************
https://kw.com/agent/UPA-6760325820224679936-7
Page load happened
44057 https://kw.com/agent/UPA-6760325820224679936-7
***********************************
https://kw.com/agent/UPA-6753348963902144512-5
Page load happened
44058 https://kw.com/agent/UPA-6753348963902144512-5
***********************************
https://kw.com/agent/UPA-6587385403447066627-5
Page load happened
44059 https://kw.com/agent/UPA-658738540

Page load happened
44106 https://kw.com/agent/UPA-6674046324024852480-6
***********************************
https://kw.com/agent/UPA-6678817338043097088-2
Page load happened
44107 https://kw.com/agent/UPA-6678817338043097088-2
***********************************
https://kw.com/agent/UPA-6587385338156019713-8
Page load happened
44108 https://kw.com/agent/UPA-6587385338156019713-8
***********************************
https://kw.com/agent/UPA-6592998104520761351-4
Page load happened
44109 https://kw.com/agent/UPA-6592998104520761351-4
***********************************
https://kw.com/agent/UPA-6587385361036644355-0
Page load happened
44110 https://kw.com/agent/UPA-6587385361036644355-0
***********************************
https://kw.com/agent/UPA-6587385243053903873-8
Page load happened
44111 https://kw.com/agent/UPA-6587385243053903873-8
***********************************
https://kw.com/agent/UPA-6762846959719632896-9
Page load happened
44112 https://kw.com/agent/UPA-6762846959719632896-

Page load happened
44159 https://kw.com/agent/UPA-6587385453945180161-0
***********************************
https://kw.com/agent/UPA-6587385259170422787-9
Page load happened
44160 https://kw.com/agent/UPA-6587385259170422787-9
***********************************
https://kw.com/agent/UPA-6778821566531084288-9
Page load happened
44161 https://kw.com/agent/UPA-6778821566531084288-9
***********************************
https://kw.com/agent/UPA-6819243359127396352-7
Page load happened
44162 https://kw.com/agent/UPA-6819243359127396352-7
***********************************
https://kw.com/agent/UPA-6754798552452317184-8
Page load happened
44163 https://kw.com/agent/UPA-6754798552452317184-8
***********************************
https://kw.com/agent/UPA-6795099726471106560-8
Page load happened
44164 https://kw.com/agent/UPA-6795099726471106560-8
***********************************
https://kw.com/agent/UPA-6587385299656134657-9
Page load happened
44165 https://kw.com/agent/UPA-6587385299656134657-

Timeout happened no page load
44212 https://kw.com/agent/UPA-6846842764825440256-3
***********************************
https://kw.com/agent/UPA-6792818867500720128-8
Timeout happened no page load
44213 https://kw.com/agent/UPA-6792818867500720128-8
***********************************
https://kw.com/agent/UPA-6587385342553694212-2
Page load happened
44214 https://kw.com/agent/UPA-6587385342553694212-2
***********************************
https://kw.com/agent/UPA-6587385339442720775-4
Page load happened
44215 https://kw.com/agent/UPA-6587385339442720775-4
***********************************
https://kw.com/agent/UPA-6724690552045522944-7
Page load happened
44216 https://kw.com/agent/UPA-6724690552045522944-7
***********************************
https://kw.com/agent/UPA-6587384954473148422-0
Page load happened
44217 https://kw.com/agent/UPA-6587384954473148422-0
***********************************
https://kw.com/agent/UPA-6587385300969164807-0
Page load happened
44218 https://kw.com/agent/UP

...

Page load happened
44742 https://kw.com/agent/UPA-6587385162207318019-8
***********************************
https://kw.com/agent/UPA-6796532947894697984-6
Page load happened
44743 https://kw.com/agent/UPA-6796532947894697984-6
***********************************
https://kw.com/agent/UPA-6592235572357779460-7
Page load happened
44744 https://kw.com/agent/UPA-6592235572357779460-7
***********************************
https://kw.com/agent/UPA-6587385432292134912-5
Page load happened
44745 https://kw.com/agent/UPA-6587385432292134912-5
***********************************
https://kw.com/agent/UPA-6834552922557333504-9
Page load happened
44746 https://kw.com/agent/UPA-6834552922557333504-9
***********************************
https://kw.com/agent/UPA-6648254360042172416-9
Page load happened
44747 https://kw.com/agent/UPA-6648254360042172416-9
***********************************
https://kw.com/agent/UPA-6734126781506949120-1
Page load happened
44748 https://kw.com/agent/UPA-6734126781506949120-

Page load happened
44795 https://kw.com/agent/UPA-6587385210450616320-6
***********************************
https://kw.com/agent/UPA-6587385315124252672-8
Page load happened
44796 https://kw.com/agent/UPA-6587385315124252672-8
***********************************
https://kw.com/agent/UPA-6812839784979083264-9
Page load happened
44797 https://kw.com/agent/UPA-6812839784979083264-9
***********************************
https://kw.com/agent/UPA-6802980436082868224-1
Page load happened
44798 https://kw.com/agent/UPA-6802980436082868224-1
***********************************
https://kw.com/agent/UPA-6587385251115782145-2
Page load happened
44799 https://kw.com/agent/UPA-6587385251115782145-2
***********************************
https://kw.com/agent/UPA-6587385354454077445-7
need to close driver
new driver initialize****************
Page load happened
44800 https://kw.com/agent/UPA-6587385354454077445-7
***********************************
https://kw.com/agent/UPA-6592247219271733250-5
Page load h

Page load happened
44848 https://kw.com/agent/UPA-6620188039901724674-0
***********************************
https://kw.com/agent/UPA-6808401228529221632-1
Page load happened
44849 https://kw.com/agent/UPA-6808401228529221632-1
***********************************
https://kw.com/agent/UPA-6771197301150547968-3
Page load happened
44850 https://kw.com/agent/UPA-6771197301150547968-3
***********************************
https://kw.com/agent/UPA-6783065179103170560-8
Page load happened
44851 https://kw.com/agent/UPA-6783065179103170560-8
***********************************
https://kw.com/agent/UPA-6819015336573763584-2
Page load happened
44852 https://kw.com/agent/UPA-6819015336573763584-2
***********************************
https://kw.com/agent/UPA-6812810148792676352-0
Timeout happened no page load
44853 https://kw.com/agent/UPA-6812810148792676352-0
***********************************
https://kw.com/agent/UPA-6734247167857188864-9
Page load happened
44854 https://kw.com/agent/UPA-673424716

Page load happened
44901 https://kw.com/agent/UPA-6587385307001491464-3
***********************************
https://kw.com/agent/UPA-6609393703949840388-6
Page load happened
44902 https://kw.com/agent/UPA-6609393703949840388-6
***********************************
https://kw.com/agent/UPA-6587385364807389194-7
Page load happened
44903 https://kw.com/agent/UPA-6587385364807389194-7
***********************************
https://kw.com/agent/UPA-6587385364731891717-6
Page load happened
44904 https://kw.com/agent/UPA-6587385364731891717-6
***********************************
https://kw.com/agent/UPA-6587385238090936322-8
Page load happened
44905 https://kw.com/agent/UPA-6587385238090936322-8
***********************************
https://kw.com/agent/UPA-6816086417726468096-4
Page load happened
44906 https://kw.com/agent/UPA-6816086417726468096-4
***********************************
https://kw.com/agent/UPA-6587385236218245123-6
Page load happened
44907 https://kw.com/agent/UPA-6587385236218245123-

Page load happened
44954 https://kw.com/agent/UPA-6587385303436267520-5
***********************************
https://kw.com/agent/UPA-6806664164308516864-6
Page load happened
44955 https://kw.com/agent/UPA-6806664164308516864-6
***********************************
https://kw.com/agent/UPA-6672116963575963648-8
Page load happened
44956 https://kw.com/agent/UPA-6672116963575963648-8
***********************************
https://kw.com/agent/UPA-6709560384518393856-0
Page load happened
44957 https://kw.com/agent/UPA-6709560384518393856-0
***********************************
https://kw.com/agent/UPA-6587385273983864839-8
Timeout happened no page load
44958 https://kw.com/agent/UPA-6587385273983864839-8
***********************************
https://kw.com/agent/UPA-6592957826888962049-6
Page load happened
44959 https://kw.com/agent/UPA-6592957826888962049-6
***********************************
https://kw.com/agent/UPA-6739185568736415744-1
Page load happened
44960 https://kw.com/agent/UPA-673918556